# Functions Definitions

In [5]:
import requests
import time
import json
import urllib.request

COMFY_HOST = "127.0.0.1:8188"

def check_server(url, retries=500, delay=50):
    """
    Check if a server is reachable via HTTP GET request

    Args:
    - url (str): The URL to check
    - retries (int, optional): The number of times to attempt connecting to the server. Default is 50
    - delay (int, optional): The time in milliseconds to wait between retries. Default is 500

    Returns:
    bool: True if the server is reachable within the given number of retries, otherwise False
    """

    for i in range(retries):
        try:
            response = requests.get(url)

            # If the response status code is 200, the server is up and running
            if response.status_code == 200:
                print(f"runpod-worker-comfy - API is reachable")
                return True
        except requests.RequestException as e:
            # If an exception occurs, the server may not be ready
            pass

        # Wait for the specified delay before retrying
        time.sleep(delay / 1000)

    print(
        f"runpod-worker-comfy - Failed to connect to server at {url} after {retries} attempts."
    )
    return False

def queue_workflow(workflow):
    """
    Queue a workflow to be processed by ComfyUI

    Args:
        workflow (dict): A dictionary containing the workflow to be processed

    Returns:
        dict: The JSON response from ComfyUI after processing the workflow
    """

    # The top level element "prompt" is required by ComfyUI
    data = json.dumps({"prompt": workflow}).encode("utf-8")

    req = urllib.request.Request(f"http://{COMFY_HOST}/prompt", data=data)
    return json.loads(urllib.request.urlopen(req).read())


def get_history(prompt_id):
    """
    Retrieve the history of a given prompt using its ID

    Args:
        prompt_id (str): The ID of the prompt whose history is to be retrieved

    Returns:
        dict: The history of the prompt, containing all the processing steps and results
    """
    with urllib.request.urlopen(f"http://{COMFY_HOST}/history/{prompt_id}") as response:
        return json.loads(response.read())

# 1. Check if ComfyUI is running

In [26]:
check_server(f"http://{COMFY_HOST}")

runpod-worker-comfy - API is reachable


True

# 2. Queue a workflow

In [31]:
# workflow = json.load(open("ready-to-use/Reindeer.json"))["input"]["workflow"]
workflow = json.load(open("3I2V.json"))
queued_workflow = queue_workflow(workflow)
prompt_id = queued_workflow["prompt_id"]
print(f"Prompt ID: {prompt_id}")

Prompt ID: 0967ce1f-8b38-4d20-b3c7-7ec30ed04f4c


# 3. Get the history of the workflow

In [32]:
history = get_history(prompt_id)
if prompt_id in history and history[prompt_id].get("outputs"):
    output = history[prompt_id].get("outputs")
    print(f"Output: {output}")
else:
    print(history)
    print("No output found for the given prompt ID. Maybe the workflow is still processing?")


Output: {'176': {'text': ['69x832x480']}, '123': {'text': ['1x1250x1250']}, '164': {'text': ['9x624x624']}, '78': {'text': ['1x1280x720']}, '30': {'gifs': [{'filename': 'WanVideoWrapper_Skyreels-A2_00001.mp4', 'subfolder': '', 'type': 'temp', 'format': 'video/h264-mp4', 'frame_rate': 16.0, 'workflow': 'WanVideoWrapper_Skyreels-A2_00001.png', 'fullpath': '/home/money/BakstersShowcaseEndpoint/ComfyUIProdMirror/temp/WanVideoWrapper_Skyreels-A2_00001.mp4'}]}, '147': {'text': ['81x832x480']}}


In [30]:
history = get_history(prompt_id)
if prompt_id in history and history[prompt_id].get("outputs"):
    output = history[prompt_id].get("outputs")
    print(f"Output: {output}")
else:
    print(history)
    print("No output found for the given prompt ID. Maybe the workflow is still processing?")


Output: {'9': {'images': [{'filename': 'ComfyUI_00021_.png', 'subfolder': '', 'type': 'output'}]}}


# 4. Output Processing

In [ ]:
def process_output_images(outputs, job_id, return_type):
    """
    This function takes the "outputs" from image generation and the job ID,
    then determines the correct way to return the image, either as a direct URL
    to an AWS S3 bucket or as a base64 encoded string, depending on the
    environment configuration.

    Args:
        outputs (dict): A dictionary containing the outputs from image generation,
                        typically includes node IDs and their respective output data.
        job_id (str): The unique identifier for the job.

    Returns:
        dict: A dictionary with the status ('success' or 'error') and the message,
              which is either the URL to the image in the AWS S3 bucket or a base64
              encoded string of the image. In case of error, the message details the issue.

    The function works as follows:
    - It first determines the output path for the images from an environment variable,
      defaulting to "/comfyui/output" if not set.
    - It then iterates through the outputs to find the filenames of the generated images.
    - After confirming the existence of the image in the output folder, it checks if the
      AWS S3 bucket is configured via the BUCKET_ENDPOINT_URL environment variable.
    - If AWS S3 is configured, it uploads the image to the bucket and returns the URL.
    - If AWS S3 is not configured, it checks if Cloudinary configuration is available.
    - If Cloudinary is configured, it uploads the image to Cloudinary and returns the URL.
    - If neither AWS S3 nor Cloudinary is configured, it encodes the image in base64 and returns the string.
    - If the image file does not exist in the output folder, it returns an error status
      with a message indicating the missing image file.
    """

    # The path where ComfyUI stores the generated images
    COMFY_OUTPUT_PATH = os.environ.get("COMFY_OUTPUT_PATH", "/comfyui/output")

    output_images = {}

    for node_id, node_output in outputs.items():
        if "images" in node_output:
            for image in node_output["images"]:
                output_images = os.path.join(image["subfolder"], image["filename"])

    print(f"runpod-worker-comfy - image generation is done")

    # expected image output folder
    local_image_path = f"{COMFY_OUTPUT_PATH}/{output_images}"

    print(f"runpod-worker-comfy - {local_image_path}")

    # The image is in the output folder
    if os.path.exists(local_image_path):
        if os.environ.get("BUCKET_ENDPOINT_URL", False) and return_type == "AWS_S3_URL":
            # URL to image in AWS S3
            image = rp_upload.upload_image(job_id, local_image_path)
            print(
                "runpod-worker-comfy - the image was generated and uploaded to AWS S3"
            )
        elif (os.environ.get("CLOUDINARY_CLOUD_NAME") and 
              os.environ.get("CLOUDINARY_API_KEY") and 
              os.environ.get("CLOUDINARY_API_SECRET")) and return_type == "Cloudinary_URL":
            # Configure Cloudinary
            cloudinary.config(
                cloud_name=os.environ.get("CLOUDINARY_CLOUD_NAME"),
                api_key=os.environ.get("CLOUDINARY_API_KEY"),
                api_secret=os.environ.get("CLOUDINARY_API_SECRET"),
                secure=True
            )
            
            # Upload to Cloudinary with a unique public_id based on job_id
            upload_result = cloudinary.uploader.upload(
                local_image_path,
                resource_type="auto",
                public_id=f"comfyui-{job_id}"
            )
            
            # Get the secure URL from the upload result
            image = {
                "public_id": f"comfyui-{job_id}",
                "secure_url": upload_result["secure_url"]
            }
            
            print(
                "runpod-worker-comfy - the image was generated and uploaded to Cloudinary"
            )
        else:
            # base64 image
            image = base64_encode(local_image_path)
            print(
                "runpod-worker-comfy - the image was generated and converted to base64"
            )

        return {
            "status": "success",
            "message": image,
        }
    else:
        print("runpod-worker-comfy - the image does not exist in the output folder")
        return {
            "status": "error",
            "message": f"the image does not exist in the specified output folder: {local_image_path}",
        }